## **Extracting Transforming Load (ETL)**

En este notebook se planea procesar los datos encontrados en los archivos .json. Con este fin vamos a usar distintas librerias que nos permitan obtener una base de datos limpia. Es de tener en cuenta que se usaran la menor cantidad de celdas de accion con el fin de hacer nuestro codigo lo mas corto y simple posible. De ser necesario, las interacciones tendran su respectivo comentario que nos ayudara a entender el codigo.

Nuestro codigo se repartira en distintas secciones teniendo una similitud con el patron Modelo - Vista - Controlador (MVC):

* Celda de librerias.
* Celda de acciones.
* Celda de vistas.

## 1. Librerias
    
    Esta Celda nos ayudara a traer todas las librerias necesarias con el fin de completar el ejercicio

In [5]:
#Importamos las librerias necesarias para el ejercicio
import json, ast, re, gzip, jsonlines
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq


## 2. Acciones
    Esta celda nos ayudara a crear todas las funciones necesarias.

In [6]:
#Funcion que toma un json con dobles comillas y lo convierte en dataframe
def json_double_a_df(json_file): 
    data = []

# Abrir el archivo comprimido y leer su contenido línea por línea
    with gzip.open(json_file, 'rb') as f:
        for line in f:
            json_line = line.decode('utf-8')
            data.append(json.loads(json_line))

    dataframe = pd.DataFrame(data)

    return dataframe


#Funcion que toma un json con comillas simples y lo convierte en dataframe
def json_simple_a_df(json_file): 

    data = []
    
    for i in gzip.open(json_file): # creo un buble para recorrer el dataset 
        data.append(ast.literal_eval(i.decode("utf-8")))

    dataframe = pd.DataFrame(data)
    
    return dataframe


#Funcion que crea diccionarios para poder contar los tipos de datos que se encuentran en cada dataframe
def diccionario_tipos_datos(dataframe_file):

    dicc_col = {}
    
    for columna in dataframe_file:
        conteo_tipos = dataframe_file[columna].apply(type).value_counts().to_dict()
        dicc_col[columna] = {str(tipo): conteo for tipo, conteo in conteo_tipos.items()}

    return dicc_col

#Funcion que convierte los datos a flotantes si son numeros o ceros si son de otro tipo

def cambio_a_float(dato):

    if pd.isna(dato):
        return 0.0
    
    try:
        return float(dato)
    except(ValueError, TypeError):
        return 0.0

#Funcion que convierte los datos a Datetime y devuelve None en caso de ser otro tipo

def cambio_a_datetime(dato):

    if pd.isna(dato):
        return None

    try:
        return pd.to_datetime(dato)
    except ValueError:
        return None
    
#Funcion que abre las sublistas dentro de un dataset y devuelve el dataset completo.

def abrir_lista(dataframe_file,nombre_col):
    
    df_aux = dataframe_file.explode(nombre_col)
    df_normal = pd.json_normalize(df_aux[nombre_col].dropna())

    df_aux.reset_index(inplace=True)
    df_normal.reset_index(inplace=True)
    dataframe_file = pd.concat([df_aux,df_normal],axis=1)
    dataframe_file.dropna(inplace=True)
    
    return dataframe_file


    
    

## 3. Vistas

## 3.1.  Obtencion de datos
    En esta seccion nos enfocamos en transformar los datos del formato original (.json) a un formato Dataframe de pandas con el fin de poder observar, eliminar, transformar y demas procesos que nos permitan una mejor visualizacion y optimizacion de dichos datos.

    Los procesos a aplicar son:
* **Transformacion de .json a Dataframe**

* **Eliminacion de columnas**

* **Eliminacion de valores Null/Nan en el Dataset steam_games**

* **Reinicio de indices**

* **Casting de datos a formatos correspondientes**

* **Apertura de datos tipo lista dentro del dataset**


## 3.1.1 ETL para el Dataset "Steam Games"

In [21]:
#Convertimos los datasets de formato json a formato pandas

steam_games_data = json_double_a_df("data\\steam_games.json.gz")
print("1. Dataset tranformados de .Json a Dataframe")

#Eliminamos columnas no necesarias para el ejercicio

columns_drop = ["app_name"]
steam_games_data.drop(columns_drop,axis=1,inplace=True) #drop col en steam_games
print(f"2. steam_games_data: Columnas eliminadas {columns_drop}")


#Eliminamos los NAN o nulls de la columna "genres"

steam_games_data.dropna(subset=["genres"],inplace=True)
print(f"3. steam_games_data: Filas con valores nulos eliminados (genres)")

#Reiniciamos los indices
steam_games_data.reset_index(inplace=True)
steam_games_data.drop(columns="index",inplace=True)
print(f"4. steam_games_data: indices reiniciados")

#Convertimos los datos de la columna "steam_games_data["price"]" a float
steam_games_data["price"] = steam_games_data["price"].apply(cambio_a_float)
print(f"5. steam_games_data: Casting a float steam_games_data['price']")


#Convertimos la columna "steam_games_data["release_date"]" a Datetime e introducimos la columna anio dentro del dataframe
steam_games_data["release_date"] = steam_games_data["release_date"].apply(cambio_a_datetime)
print(f"6. steam_games_data: Casting a datetime steam_games_data['release_date']")

steam_games_data["release_year"] = steam_games_data["release_date"].dt.year
print(f"7. steam_games_data: Columna creada steam_games_data['release_year']")


#Abrimos la lista dentro de los datasets
steam_games_data = steam_games_data.explode("genres")
print(f"8. steam_games_data: Apertura de listas steam_games_data['genres']")







1. Dataset tranformados de .Json a Dataframe
2. steam_games_data: Columnas eliminadas ['app_name']
3. steam_games_data: Filas con valores nulos eliminados (genres)
4. steam_games_data: indices reiniciados
5. steam_games_data: Casting a float steam_games_data['price']


C:\Users\andre\AppData\Local\Temp\ipykernel_2952\562530017.py:60: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(dato)


6. steam_games_data: Casting a datetime steam_games_data['release_date']
7. steam_games_data: Columna creada steam_games_data['release_year']
8. steam_games_data: Apertura de listas steam_games_data['genres']


## 3.1.2. ETL para el Dataset "User Reviews"

In [9]:
#Convertimos los datasets de formato json a formato pandas
user_reviews_data = json_simple_a_df("data\\user_reviews.json.gz")
print("1. Dataset formateado a Dataframe")

#Abrimos la lista que contiene el dataframe
user_reviews_data = abrir_lista(user_reviews_data,"reviews")
print("2. Sublistas extraidas")

#Formateamos la fecha
user_reviews_data["posted"] = user_reviews_data["posted"].str.extract(r"Posted ([\w\s\d,]+)") #Fecha de tipo "Posted", extraemos solo la fecha
user_reviews_data["posted_date"] = user_reviews_data["posted"].apply(cambio_a_datetime) #Convertimos las fechas en formato datetime
user_reviews_data["posted_year"] = user_reviews_data["posted_date"].dt.year #Creamos y extraemos los anios
print("3. Fecha formateada y nueva columna agregada: Posted_year")

#Eliminamos columnas no necesarias para el dataset
drop_col = ["reviews","last_edited","index","posted"]
user_reviews_data.drop(columns=drop_col,index=1, inplace=True) #drop col en user_reviews
print(f"4. user_reviews_data: Columnas eliminadas {drop_col}")



1. Dataset formateado a Dataframe
2. Sublistas extraidas
3. Fecha formateada y nueva columna agregada: Posted_year
4. user_reviews_data: Columnas eliminadas ['reviews', 'last_edited', 'index', 'posted']


## 3.1.3. ETL para el Dataset "User Items"

In [10]:
#Convertimos los datasets de formato json a formato pandas
user_items_data = json_simple_a_df("data\\users_items.json.gz")
print("1. Dataset formateado a Dataframe")

#Abrimos la lista que contiene el dataframe
user_items_data = abrir_lista(user_items_data,"items")
print("2. Sublistas extraidas")

#Eliminamos columnas no necesarias para el dataset
drop_col = ["items","index"]
user_items_data.drop(columns=drop_col,index=1, inplace=True) #drop col en user_reviews
print(f"3. user_item_data: Columnas eliminadas {drop_col}")



1. Dataset formateado a Dataframe
2. Sublistas extraidas
3. user_item_data: Columnas eliminadas ['items', 'index']


## 4. Seccion de pruebas
    En esta celda verificamos que los codigos funcionen antes de ser agregados a la seccion de Obtencion de datos

In [15]:
steam_games_data = json_double_a_df("data\\steam_games.json.gz")
columns_drop = ["publisher","app_name","url","reviews_url"]
steam_games_data.drop(columns_drop,axis=1,inplace=True) 
steam_games_data.dropna(subset=["genres"],inplace=True)
steam_games_data["price"] = steam_games_data["price"].apply(cambio_a_float)
steam_games_data["release_date"] = steam_games_data["release_date"].apply(cambio_a_datetime)
steam_games_data["release_year"] = steam_games_data["release_date"].dt.year
steam_games_data.reset_index(inplace=True)
steam_games_data.drop(columns="index",inplace=True)
steam_games_data = steam_games_data.explode("genres")
steam_games_data.head()


C:\Users\andre\AppData\Local\Temp\ipykernel_2952\562530017.py:60: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(dato)


,genres,title,release_date,tags,specs,price,early_access,id,developer,release_year
0,Action,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro,2018.0
0,Casual,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro,2018.0
0,Indie,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro,2018.0
0,Simulation,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro,2018.0
0,Strategy,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro,2018.0


## 5. Revision Tipo de Datos
    Con la funcion definida anterior como "diccionario_tipos_datos(dataframe_file)" revisamos los tipos de datos que tienen cada una de las columnas dentro de nuestras bases de datos.

In [16]:
diccionario_tipos_datos(steam_games_data)

{'genres': {"<class 'str'>": 71554},
 'title': {"<class 'str'>": 71552, "<class 'float'>": 2},
 'release_date': {"<class 'pandas._libs.tslibs.timestamps.Timestamp'>": 71011,
  "<class 'pandas._libs.tslibs.nattype.NaTType'>": 543},
 'tags': {"<class 'list'>": 71508, "<class 'float'>": 46},
 'specs': {"<class 'list'>": 70997, "<class 'float'>": 557},
 'price': {"<class 'float'>": 71554},
 'early_access': {"<class 'bool'>": 71554},
 'id': {"<class 'str'>": 71552, "<class 'float'>": 2},
 'developer': {"<class 'str'>": 71204, "<class 'float'>": 350},
 'release_year': {"<class 'float'>": 71554}}

In [17]:
diccionario_tipos_datos(user_reviews_data)

{'user_id': {"<class 'str'>": 59276},
 'user_url': {"<class 'str'>": 59276},
 'funny': {"<class 'str'>": 59276},
 'item_id': {"<class 'str'>": 59276},
 'helpful': {"<class 'str'>": 59276},
 'recommend': {"<class 'bool'>": 59276},
 'review': {"<class 'str'>": 59276},
 'posted_date': {"<class 'pandas._libs.tslibs.timestamps.Timestamp'>": 49161,
  "<class 'pandas._libs.tslibs.nattype.NaTType'>": 10115},
 'posted_year': {"<class 'float'>": 59276}}

In [18]:
diccionario_tipos_datos(user_items_data)

{'user_id': {"<class 'str'>": 5136658},
 'items_count': {"<class 'int'>": 5136658},
 'steam_id': {"<class 'str'>": 5136658},
 'user_url': {"<class 'str'>": 5136658},
 'item_id': {"<class 'str'>": 5136658},
 'item_name': {"<class 'str'>": 5136658},
 'playtime_forever': {"<class 'float'>": 5136658},
 'playtime_2weeks': {"<class 'float'>": 5136658}}

## 6. Transformacion de Dataframe a Csv

In [20]:
steam_games_data.to_parquet("data\\steam_game_data",index=False)
print(f"1. Dataset Creado: steam_games_data.parquet")

user_items_data.to_parquet("data\\user_items_data",index=False)
print(f"2. Dataset Creado: user_items_data.parquet")

user_reviews_data.to_parquet("data\\user_reviews_data",index=False)
print(f"3. Dataset Creado: user_reviews_data.parquet")


1. Dataset Creado: steam_games_data.parquet
2. Dataset Creado: user_items_data.parquet
3. Dataset Creado: user_reviews_data.parquet
